# Lendo dados experimentais


In [ ]:
import numpy as np
import pandas as pd

from plot import plot_tanks

exp_data = pd.read_csv("../dados.csv")

F_exp = exp_data["vazao"].to_numpy() * 2
h1_exp = np.clip(exp_data["h1"].to_numpy(), 0.0, None)
h2_exp = np.clip(exp_data["h2"].to_numpy(), 0.0, None)
t = np.arange(len(F_exp))

print(exp_data.head())

plot_tanks(t, [F_exp, h1_exp, h2_exp], ["F (exp)", "h1 (exp)", "h2 (exp)"])


# Definindo sistema


In [2]:
import numpy as np
from scipy.integrate import solve_ivp

from typings import Bounds


def F(t: float | int):
    return F_exp[int(t)]


def edo(t, Y, cv1, cv2):
    # Constantes
    a2 = 10 * 15  # cm^2
    a1 = np.pi * (11 / 2) ** 2  # cm^2
    # a2 = 160.0
    # a1 = 95.03

    # Dependent variables
    h2, h1 = Y[0], Y[1]

    # Tanks limits
    h2 = np.where(h2 < 0.0, 0.0, h2)
    h1 = np.where(h1 < 0.0, 0.0, h1)

    # Equations
    dh2dt = (F(t) / a2) - ((cv2 * np.sqrt(h2)) / a2)
    dh1dt = ((cv2 * np.sqrt(h2)) / a1) - ((cv1 * np.sqrt(h1)) / a1)
    return [dh2dt, dh1dt]


def simulate(c1, c2):
    sol = solve_ivp(
        edo,
        [t[0], t[-1]],
        [h1_exp[0], h2_exp[0]],
        t_eval=t,
        method="RK45",
        args=(c1, c2),
    )
    return sol.y


bounds: Bounds = [(0, 8), (0, 8)]  # Limites para Cv1 e Cv2


# Definindo função objetivo


In [3]:
import concurrent.futures


def loss(C):
    c1, c2 = C
    h2_sim, h1_sim = simulate(c1, c2)
    total_loss = np.mean(np.square(h1_exp - h1_sim)) + np.mean(
        np.square(h2_exp - h2_sim)
    )

    # print(f"c1: {c1}, c2: {c2}, loss: {total_loss}")
    return total_loss


# Visualizando a função objetivo
c1_vals = np.linspace(bounds[0][0], bounds[0][1], 200)  # Intervalo de valores para c1
c2_vals = np.linspace(bounds[1][0], bounds[1][1], 200)  # Intervalo de valores para c2
C1, C2 = np.meshgrid(c1_vals, c2_vals)

# Calculando em paralelo os valores para maior velocidade
combinations = [
    (C1[i, j], C2[i, j]) for i in range(C1.shape[0]) for j in range(C1.shape[1])
]
Z = np.zeros_like(C1)

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(loss, combinations))

for idx, result in enumerate(results):
    i, j = divmod(idx, C1.shape[1])
    Z[i, j] = result


In [ ]:
from plot import plot_space

# Plotando o gráfico
plot_space(C1, C2, Z, None, "objetive_function.png")


# Implementação do algorítimo genético clássico


In [ ]:
from genetic_utils import crossover, initialize_population, mutate, tournament_selection
from typings import GA_params, Population

# Fixando seed
np.random.seed(42)


# Função de avaliação do fitness (quanto menor o valor da função, melhor)
def fitness(individual):
    x, y = individual
    return -loss([x, y])  # Negativo porque estamos minimizando


# Algoritmo Genético
def genetic_algorithm(params: GA_params, fitness_func):
    # Inicializar a população
    population = initialize_population(params, fitness_func)
    plot_space(C1, C2, Z, population, "generation_0.png")
    best_individual = max(population, key=lambda ind: ind["fitness"])

    for generation in range(params["num_generations"]):
        new_population: Population = []

        while len(new_population) < params["pop_size"]:
            # Selecionar pais
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)

            # Realizar crossover
            child1, child2 = crossover(parent1, parent2, fitness_func, params)

            # Realizar mutação
            child1 = mutate(child1, fitness_func, params)
            child2 = mutate(child2, fitness_func, params)

            # Adicionar filhos à nova população
            new_population.append(child1)
            if len(new_population) < params["pop_size"]:
                new_population.append(child2)

        population = new_population
        best_new_individual = max(population, key=lambda ind: ind["fitness"])
        if best_individual["fitness"] < best_new_individual["fitness"]:
            best_individual = best_new_individual

        plot_space(C1, C2, Z, population, f"generation_{generation + 1}.png")
        print(
            f"Geração {generation + 1}: Melhor solução = {best_individual['chromosome']}, Melhor fitness = {-best_individual['fitness']}"
        )

    return best_individual


# Definir os parâmetros do algoritmo
genetic_alg_params: GA_params = {
    "pop_size": 50,
    "num_generations": 25,
    "bounds": bounds,
    "mutation_rate": 0.1,
    "mutation_strength": 0.1,
    "crossover_rate": 0.8,
}

# Executar o algoritmo genético
best_individual = genetic_algorithm(genetic_alg_params, fitness)

print(f"Melhor solução encontrada: {best_individual["chromosome"]}")
print(f"Valor da função objetivo na melhor solução: {-best_individual['fitness']}")


In [ ]:
cv1_salomão = 3.582
cv2_salomão = 2.869

h2_opt, h1_opt = simulate(
    best_individual["chromosome"][0], best_individual["chromosome"][1]
)
h2_chute, h1_chute = simulate(8, 0.5)
# h2_opt, h1_opt = simulate(cv1_salomão, cv2_salomão)

plot_tanks(
    t,
    [h1_exp, h2_exp, h1_opt, h2_opt],
    [
        "h1 (dados experimentais)",
        "h2 (dados experimentais)",
        "h1 (simulação após otimização)",
        "h2 (simulação após otimização)",
    ],
    "tanks_opt.png",
)

plot_tanks(
    t,
    [h1_exp, h2_exp, h2_chute, h2_opt],
    [
        "h1 (dados experimentais)",
        "h2 (dados experimentais)",
        "h1 (simulação antes da otimização)",
        "h2 (simulação antes da otimização)",
    ],
    "tanks_chute.png",
)
